**HOMEWORK 7**

**Author:** Alberto Polo

**NYUID:** N19683411

*Description:* Apply the contraction mapping theorem to a risk-neutral asset pricing model, first with homogeneous beliefs, then with heterogenous beliefs

**Exercise 1**

I'm solving this homework in Julia. Thus, as a first step, I create a Julia type called *primitives* which holds all primitives of the model, namely the discount factor $\beta$, the number of states, the dividend payment at each state and the stochastic matrix for states. I also create a second type called *pricefn* to hold the equilibrium price vector.

In [1]:
type primitives
    beta::Float64 # discount factor
    nstates::Int64 # cardinality of S, assumed finite
    d::Vector{Float64} # dividend flow for each state i in S
    Q::Matrix{Float64} # stochastic matrix for states
end

type pricefn
    p::Vector{Float64} # equilibrium price for each state i in S
end

Next I write a function called *solve_rneutral* which takes as input a specification of the primitives of the model, solves for the equilibrium price vector and returns its values. I use keyword arguments for solution algorithm details such as the initial price vector, the maximum number of iterations and the tolerance level to establish convergence (here the sup-norm corresponds to the maximum absolute value *maxabs*).

Notice that I can write the contraction compactly in vector form as $Tp = \beta Q (d + p)$ since, for each state, the equilibrium price is the discounted expected value of the asset next period, which itself comprises dividend payment and resale price.

In [2]:
function solve_rneutral(m::primitives; f = pricefn(zeros(m.nstates)), maxiter = 1000, tol = 1e-4)
    
    pcurr = f.p
    beta, d, Q = m.beta, m.d, m.Q

    for i in 1:maxiter

        pnext = beta * (Q * (d + pcurr))
        
        err = maxabs(pnext - pcurr)
        copy!(f.p, pnext)
        
        if i % 100 == 0; @printf("Iteration %d, distance = %1.4f\n", i, err); end
        
        if err < tol; break; end
        
    end
    
    @printf("The equilibrium price function is: p(NG) = %1.4f, p(MR) = %1.4f, p(SR) = %1.4f\n", f.p[1], f.p[2], f.p[3])
    
    return f
end 

solve_rneutral (generic function with 1 method)

In order to run the first test of the code I write a function which assigns the primitives (and checks that I write a proper stochastic matrix!).

In [3]:
function test1()
    
    beta = .98
    nstates = 3
    d = [1.; 0.; 0.]
    
    Q = [.971 .029 0.;
        .145 .778 .077;
        0. .508 .492]
    
    if (sum(Q, 2) != ones(nstates, 1)); @printf("Q is not a stochastic matrix!"); end
    
    return primitives(beta, nstates, d, Q)
end

test1 (generic function with 1 method)

Then I can finally run the contraction algorithm to solve for the approximate equilibrium vector.

In [7]:
eqp1 = solve_rneutral(test1());

Iteration 100, distance = 0.1078
Iteration 200, distance = 0.0143
Iteration 300, distance = 0.0019
Iteration 400, distance = 0.0003
The equilibrium price function is: p(NG) = 40.7549, p(MR) = 35.9560, p(SR) = 34.5671


As expected the equilibrium price of the asset is highest in the normal growth state (NG) (when the asset also pays dividends). In terms of dividends the two recession states are equivalent. However the mild recession state (MR) allows a positive probability of direct transition to the NG state, while the severe recession state (SR) does not allow that. Hence the price in the MR state is higher than in the SR state.

For the second experiment we use a different specification of the stochastic matrix. The only difference is that, once in the NG state, there is a positive probability of transiting to the SR state directly, and a lower probability of remaining in the NG state.

In [8]:
function test2()
    
    beta = .98
    nstates = 3
    d = [1.; 0.; 0.]
    
    Q = [.871 .029 0.1;
        .145 .778 .077;
        0. .508 .492]
    
    if (sum(Q, 2) != ones(nstates, 1)); @printf("Q is not a stochastic matrix!"); end
    
    return primitives(beta, nstates, d, Q)
end

test2 (generic function with 1 method)

In [9]:
solve_rneutral(test2());

Iteration 100, distance = 0.0597
Iteration 200, distance = 0.0079
Iteration 300, distance = 0.0011
Iteration 400, distance = 0.0001
The equilibrium price function is: p(NG) = 23.4963, p(MR) = 21.0940, p(SR) = 20.2791


The change in the stochastic matrix has a large impact on the equilibrium price vector. Clearly, the ranking of prices across states is unchanged. However the entire level is significantly lower than before. With risk-neutral pricing, this happens because the process will spend more time in states where the asset pays no dividend, thus being intrinsically less able to generate income flows than before. This is reflected in a lower equilibrium price.

**Exercise 2**

The proof is similar to the homogenous belief case. Let $d\in b\mathbb{R}^S$.

*Step 1:* prove that $T: b\mathbb{R}^S \rightarrow b\mathbb{R}^S$

Let $p \in b\mathbb{R}^S$ and $x \in S$. Notice that, if we let $b_i = 0$ for all $i$, the inequality suggested in the exercise implies $\mid \max_i a_i\mid \leq \max_i \mid a_i\mid$. Then

\begin{equation}
\mid Tp(x) \mid = \beta \mid \max_i \left\{ \sum_y\left[ d(y) + p(y)\right]q_i(x, y)\right\} \mid
\end{equation}

\begin{equation}
\leq \beta \max_i \left\{ \mid \sum_y\left[ d(y) + p(y)\right]q_i(x, y)\mid\right\}
\end{equation}

Take any $i$. Then

\begin{equation}
\mid \sum_y\left[ d(y) + p(y)\right]q_i(x, y)\mid \leq \sum_y\mid d(y) + p(y)\mid q_i(x, y) \leq \sum_y \left[\lVert d \rVert_\infty + \lVert p \rVert_\infty\right] q_i(x, y) = \lVert d \rVert_\infty + \lVert p \rVert_\infty < \infty \mbox{ by assumption}
\end{equation}

Notice that the bound is independent of $i$, which implies

\begin{equation}
\mid Tp(x) \mid \leq \beta \max_i \left\{ \mid \sum_y\left[ d(y) + p(y)\right]q_i(x, y)\mid\right\} \leq \beta\left[\lVert d \rVert_\infty + \lVert p \rVert_\infty\right]
\end{equation}

Since $x$ is arbitrary, $\lVert Tp \rVert_\infty < \infty$ i.e. $Tp \in b\mathbb{R}^S$.

*Step 2:* prove that $T$ is a uniform contraction on $(b\mathbb{R}^S, d_\infty$)

Let $p, p' \in b\mathbb{R}^S$ and $x \in S$. Then

\begin{equation}
\mid Tp(x) - Tp'(x) \mid = \beta \mid \max_i \left\{ \sum_y\left[ d(y) + p(y)\right]q_i(x, y)\right\} - \max_i \left\{ \sum_y\left[ d(y) + p'(y)\right]q_i(x, y)\right\} \mid
\end{equation}

\begin{equation}
\leq \beta \max_i \left\{ \mid \sum_y\left[ d(y) + p(y)\right]q_i(x, y) - \sum_y\left[ d(y) + p'(y)\right]q_i(x, y) \mid \right\} = \beta \max_i \left\{ \mid \sum_y\left[ p(y) - p'(y)\right]q_i(x, y)\mid \right\}
\end{equation}

were the inequality is obtained using the result suggested in the exercise.

Take any $i$. Notice that, since $b\mathbb{R}^S$ is a vector space, $p - p' \in b\mathbb{R}^S$. Then

\begin{equation}
\mid \sum_y\left[ p(y) - p'(y)\right]q_i(x, y)\mid \leq \sum_y\mid p(y) - p'(y)\mid q_i(x, y) \leq \sum_y \lVert p - p' \rVert_\infty q_i(x, y) = \lVert p - p' \rVert_\infty
\end{equation}

Again the bound is independent of $i$. Hence

\begin{equation}
\mid Tp(x) - Tp'(x) \mid \leq \beta \max_i \left\{ \mid \sum_y\left[ p(y) - p'(y)\right]q_i(x, y)\mid \right\} \leq \beta \lVert p - p' \rVert_\infty
\end{equation}

and since $x$ was arbitrary, $\lVert Tp - Tp' \rVert_\infty \leq \beta \lVert p - p' \rVert_\infty$ i.e. $T$ is a uniform contraction on $(b\mathbb{R}^S, d_\infty$)

**Proving the inequality**

Let us assume that $\sup_i a_i<\infty$ and $\sup_i b_i<\infty$. Then

\begin{equation}
sup_i a_i = sup_i (a_i - b_i + b_i) \leq sup_i (a_i - b_i) + sup_i b_i
\end{equation}

Thus

\begin{equation}
sup_i a_i - sup_i b_i \leq sup_i (a_i - b_i) \leq sup_i \mid a_i - b_i \mid
\end{equation}

We can then do the same with $b_i$ and get

\begin{equation}
sup_i b_i - sup_i a_i = -( sup_i a_i - sup_i b_i)\leq sup_i \mid a_i - b_i \mid
\end{equation}

Hence

\begin{equation}
\mid sup_i a_i - sup_i b_i \mid \leq sup_i \mid a_i - b_i \mid
\end{equation}


**Exercise 3**

For this exercise I create a new type *primitives_heterog* which holds, among other things, the two transition probabilities corresponding to the two beliefs on the evolution of the aggregate state.

In [10]:
type primitives_heterog
    beta::Float64 # discount factor
    nstates::Int64 # cardinality of S, assumed finite
    d::Vector{Float64} # dividend flow for each state i in S
    Q1::Matrix{Float64} # stochastic matrix for states of agent 1
    Q2::Matrix{Float64} # stochastic matrix for states of agent 2
end

Then I write a function that solves for the equilibrium price vector using the contraction mapping theorem. Notice that this time the contraction is written in compact form as a row-wise maximum. As a matter of fact, for each state, the price of the asset is the maximum discounted expected valuation of the asset in the following period.

In [11]:
function solve_rneutral_heterog(m::primitives_heterog; f = pricefn(zeros(m.nstates)), maxiter = 1000, tol = 1e-4)
    
    pcurr = f.p
    beta, d, Q1, Q2 = m.beta, m.d, m.Q1, m.Q2

    for i in 1:maxiter

        pnext = beta * max(Q1 * (d + pcurr),  Q2 * (d + pcurr))
        
        err = maxabs(pnext - pcurr)
        copy!(f.p, pnext)
        
        if i % 100 == 0; @printf("Iteration %d, distance = %1.4f\n", i, err); end
        
        if err < tol; break; end
        
    end
    
    @printf("The equilibrium price function is: p(NG) = %1.4f, p(MR) = %1.4f, p(SR) = %1.4f\n", f.p[1], f.p[2], f.p[3])
    
    return f
end 

solve_rneutral_heterog (generic function with 1 method)

The test function now assigns the two stochastic matrixes to the different agents.

In [12]:
function test_heterog()
    
    beta = .98
    nstates = 3
    d = [1.; 0.; 0.]
    
    Q1 = [.971 .029 0.;
        .145 .778 .077;
        0. .508 .492]
    
    Q2 = [.871 .029 0.1;
        .145 .778 .077;
        0. .508 .492]
    
    if (sum(Q1, 2) != ones(nstates, 1)); @printf("Q1 is not a stochastic matrix!"); end
    if (sum(Q2, 2) != ones(nstates, 1)); @printf("Q2 is not a stochastic matrix!"); end
    
    return primitives_heterog(beta, nstates, d, Q1, Q2)
end

test_heterog (generic function with 1 method)

In [14]:
eqp_heterog = solve_rneutral_heterog(test_heterog());

Iteration 100, distance = 0.1078
Iteration 200, distance = 0.0143
Iteration 300, distance = 0.0019
Iteration 400, distance = 0.0003
The equilibrium price function is: p(NG) = 40.7549, p(MR) = 35.9560, p(SR) = 34.5671


In [15]:
maxabs(eqp1.p - eqp_heterog.p)

0.0

Solving for the equilibrium price vector confirms what we expected: the equilibrium price vector corresponds to the price vector of the agent with the highest valuation, which in this case is agent 1